Import Libs

In [0]:
import numpy
import ogr

from matplotlib import pyplot as plt
from osgeo import gdal

In [0]:
!ls

In [0]:
!pwd

In [0]:
! git clone --recursive https://github.com/marujore/TAPDI.git

In [0]:
cd TAPDI

In [0]:
!ls

In [0]:
cd data_cube

In [0]:
!ls

In [0]:
cd imgs

In [0]:
!ls

Open Images

In [0]:
def printminmax(img):
  print('Array size: {}'.format(img.shape) )
  print('min value:{} max value:{}'.format( numpy.nanmin(img[img != -numpy.inf]) , numpy.nanmax(img[img != numpy.inf]) ))

filename_nir = '/content/TAPDI/data_cube/imgs/S2B_MSIL2A_20180922T132229_N0206_R038_T23LMG_20180922T183847_nir.tif'

### open nir
try:
  Raster = gdal.Open(filename_nir, gdal.GA_ReadOnly)
  print ("Opened {}!".format(filename_nir))
except:
  print("Error opening file {}".format(filename_nir) )
  
img_nir = Raster.GetRasterBand(1).ReadAsArray()
printminmax(img_nir)

### Plot image
plt.figure(figsize=(10, 10))
plt.imshow(img_nir,cmap = 'gray')
plt.show()

Clip tifs using shp

In [0]:
#Shapefile = '/content/TAPDI/data_cube/Grid.shp'
#VectorFormat = 'ESRI Shapefile'
#VectorDriver = ogr.GetDriverByName(VectorFormat)
#VectorDataset = VectorDriver.Open(Shapefile, 0) # 0=Read-only, 1=Read-Write
#layer = VectorDataset.GetLayer()
#FeatureCount = layer.GetFeatureCount()
#print("Feature Count:",FeatureCount)

#RasterFormat = 'GTiff'
#PixelRes = 10
#Projection = Raster.GetProjectionRef()

### Iterate through the shapefile features
#Count = 0
#for feature in layer:
#    Count += 1
#    print("Processing feature "+str(Count)+" of "+str(FeatureCount)+"...")

#    geom = feature.GetGeometryRef() 
#    minX, maxX, minY, maxY = geom.GetEnvelope() # Get bounding box of the shapefile feature

    ### Create raster
#    OutTileName = str(Count)+'.SomeTileName.tif'
#    OutTile = gdal.Warp(OutTileName, Raster, format=RasterFormat, outputBounds=[minX, minY, maxX, maxY], xRes=PixelRes, yRes=PixelRes, dstSRS=Projection, resampleAlg=gdal.GRA_NearestNeighbour, options=['COMPRESS=DEFLATE'])
#    OutTile = None # Close dataset

### Close datasets
#Raster = None
#VectorDataset.Destroy()
#print("Done.")

In [0]:
cd ..

In [0]:
!ls

In [0]:
!mv Grid.shx imgs/

In [0]:
!mv *Grid.* imgs/

In [0]:
!ls

In [0]:
cd imgs/

In [0]:
!ls

In [0]:
!for f in *.tif; do gdalwarp -of GTiff -cutline Grid.shp -crop_to_cutline  "$f" "${f%.*}_crop.tif"; done

In [0]:
!ls

In [0]:
Raster = None

filename_nir = '/content/TAPDI/data_cube/imgs/S2B_MSIL2A_20180922T132229_N0206_R038_T23LMG_20180922T183847_nir.tif'

### open nir
try:
  Raster = gdal.Open(filename_nir, gdal.GA_ReadOnly)
  print ("Opened {}!".format(filename_nir))
except:
  print("Error opening file {}".format(filename_nir) )
  
img_nir = Raster.GetRasterBand(1).ReadAsArray()
printminmax(img_nir)

### Plot image
plt.figure(figsize=(10, 10))
plt.imshow(img_nir,cmap = 'gray')
plt.show()
Raster = None

filename_nir = '/content/TAPDI/data_cube/imgs_crop/S2B_MSIL2A_20180922T132229_N0206_R038_T23LMG_20180922T183847_nir_crop.tif'

### open nir
try:
  Raster = gdal.Open(filename_nir, gdal.GA_ReadOnly)
  print ("Opened {}!".format(filename_nir))
except:
  print("Error opening file {}".format(filename_nir) )
  
img_nir = Raster.GetRasterBand(1).ReadAsArray()
printminmax(img_nir)

### Plot image
plt.figure(figsize=(10, 10))
plt.imshow(img_nir,cmap = 'gray')
plt.show()
Raster = None

AFTER PERFORM MERGE

In [0]:
img_list = [
'/content/TAPDI/data_cube/imgs_crop/S2SR-089098-20180902_M_3_nir_crop.tif',
'/content/TAPDI/data_cube/imgs_crop/S2SR-089098-20180907_M_4_nir_crop.tif',
'/content/TAPDI/data_cube/imgs_crop/S2SR-089098-20180912_M_2_nir_crop.tif',
'/content/TAPDI/data_cube/imgs_crop/S2SR-089098-20180917_M_4_nir_crop.tif',
'/content/TAPDI/data_cube/imgs_crop/S2SR-089098-20180922_M_4_nir_crop.tif',
'/content/TAPDI/data_cube/imgs_crop/S2SR-089098-20180927_M_4_nir_crop.tif'
]

mask_list = [
'/content/TAPDI/data_cube/imgs_crop/S2SR-089098-20180902_M_3_mask_crop.tif',
'/content/TAPDI/data_cube/imgs_crop/S2SR-089098-20180907_M_4_mask_crop.tif',
'/content/TAPDI/data_cube/imgs_crop/S2SR-089098-20180912_M_2_mask_crop.tif',
'/content/TAPDI/data_cube/imgs_crop/S2SR-089098-20180917_M_4_mask_crop.tif',
'/content/TAPDI/data_cube/imgs_crop/S2SR-089098-20180922_M_4_mask_crop.tif',
'/content/TAPDI/data_cube/imgs_crop/S2SR-089098-20180927_M_4_mask_crop.tif'
]
# 1 cloud free
# 2 cloud

imgs = []
score = []

for i in range( len(img_list) ):
  raster = gdal.Open(img_list[i], gdal.GA_ReadOnly)
  raster_mask = gdal.Open(mask_list[i], gdal.GA_ReadOnly)
  img = raster.GetRasterBand(1).ReadAsArray()
  img_mask = raster_mask.GetRasterBand(1).ReadAsArray()
  img_mask[img_mask == 1] = 0
  indices = numpy.nonzero(img_mask)
  #img[indices] = -99999 #numpy.nan
  imgs.append( numpy.copy(img) )
  score.append( numpy.count_nonzero(img_mask) )
  raster = None
  raster_mask = None
  print("{} Score: {}".format(i, score[i]))
  # < score better


In [0]:
# printing original list
print("The original list is : {}".format(score))

# using sorted() + index() + list comprehension
# Finding relative order of elements in list
temp = sorted(score)
res = [temp.index(i) for i in score]

# printing result
print ("The relative ordering list is : {}".format(res))

ind_ord = sorted(range(len(score)), key=lambda k: score[k])
print ("Index ordering is : {}".format( ind_ord ) )

In [0]:
### Plot image
plt.figure(figsize=(10, 10))
plt.imshow(imgs[ind_ord[0]],cmap = 'gray')
plt.show()
printminmax(imgs[ind_ord[0]])

In [0]:
imgs[ind_ord[0]][200:300,200:300] = 0

### Plot image
plt.figure(figsize=(10, 10))
plt.imshow(imgs[ind_ord[0]],cmap = 'gray')
plt.show()
printminmax(imgs[ind_ord[0]])

In [0]:
time_composed_img = numpy.copy(imgs[ind_ord[0]])

plt.figure(figsize=(10, 10))
plt.imshow(time_composed_img,cmap = 'gray')
plt.show()
printminmax(time_composed_img)

print(time_composed_img[200,200])
i = 1
tofill = 1
while i < len(imgs) and tofill > 0:
  indices = time_composed_img == 0
  #print(indices)
  
  tofill = numpy.count_nonzero(time_composed_img==0)
  print("i:{}, to fill: {}".format(i, tofill))
  time_composed_img[indices] = imgs[ind_ord[i]][indices]
  i = i + 1
  

In [0]:
### Plot image
plt.figure(figsize=(10, 10))
plt.imshow(time_composed_img,cmap = 'gray')
plt.show()
printminmax(time_composed_img)